In [15]:
import os
from dotenv import load_dotenv
assert os.getenv("OPENAI_API_KEY") is not None

import gradio as gr
from langchain_openai.chat_models import ChatOpenAI
from langchain.document_loaders import YoutubeLoader
from langchain.text_splitter import CharacterTextSplitter

from langchain_community.vectorstores.faiss import FAISS
from langchain_openai import OpenAIEmbeddings

from langchain_core.prompts import ChatPromptTemplate

from langchain_core.output_parsers import StrOutputParser

from langchain_core.documents import Document
from langchain_core.runnables import RunnablePassthrough


In [16]:
transcript_loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=HJXWpqpcHik", language="en")
transcript = transcript_loader.load()

splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
splits = splitter.split_documents(transcript)

chat_model = ChatOpenAI(model="gpt-3.5-turbo")


In [17]:
oai_embedding_model = OpenAIEmbeddings()
vector_store = FAISS.from_documents(transcript, oai_embedding_model)
faiss_retriever = vector_store.as_retriever()

In [18]:
def format_documents(docs: list[Document]):    
    return "\n\n".join([d.page_content for d in docs])

In [19]:
HUMAN_MESSAGE = """Given the context below

{context}

Please answer the following
{question}
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You're an assistant。"),
        ("human", HUMAN_MESSAGE),
    ]
)

# now note that we can use the retriever as a runnable
# and specifiying that the output of retriever should
# be fed into the `context` variable of the prompt
chain = {"context": faiss_retriever} | prompt | ChatOpenAI() | StrOutputParser()

In [20]:
chain = (
    {"context": faiss_retriever | format_documents, "question": RunnablePassthrough()}
    | prompt
    | ChatOpenAI()
    | StrOutputParser()
)

response_4 = chain.invoke("Can you summarize the video?")

print(response_4)

The video discusses creating a YouTube summarizer using AI in less than 10 lines of code. It explains the process of loading transcripts, splitting text into chunks, initializing a language model, and summarizing the transcripts using the "refine chain" method. The summary highlights the key steps involved in building the YouTube summarizer and suggests ways to enhance the summarizer further.


In [21]:
response_5 = chain.invoke("Can you summarize the video?")
print(response_5)


The video discusses creating a YouTube summarizer using AI and Lang Chain in less than 10 lines of code. It outlines the process of installing necessary libraries, loading transcripts from a YouTube video, splitting the text into smaller chunks, initializing a large language model (GPT-3.5 Turbo 16k), and summarizing the transcripts using a refine chain method. The tutorial emphasizes utilizing the OpenAI API key, setting up the model parameters, and showcasing the summarization process step by step. Overall, the video provides a comprehensive overview of building a YouTube summarizer with AI technology.


In [22]:
from langchain.schema import format_document
from langchain_core.messages import get_buffer_string
from langchain_core.runnables import RunnableParallel

from langchain.prompts.prompt import PromptTemplate

from operator import itemgetter


_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")

def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

_inputs = RunnableParallel(
    standalone_question=RunnablePassthrough.assign(
        chat_history=lambda x: get_buffer_string(x["chat_history"])
    )
    | CONDENSE_QUESTION_PROMPT
    | ChatOpenAI(temperature=0)
    | StrOutputParser(),
)
_context = {
    "context": itemgetter("standalone_question") | faiss_retriever | _combine_documents,
    "question": lambda x: x["standalone_question"],
}
conversational_qa_chain = _inputs | _context | ANSWER_PROMPT | ChatOpenAI() | StrOutputParser()

conversational_qa_chain.invoke(
    {
        "question": "Which text splitter did the author use?",
        "chat_history": [],
    }
)

'The author used the token text splitter to split the text into smaller chunks.'

In [23]:
def my_chat_function(message: str, history):
    response = conversational_qa_chain.invoke({"question": message, "chat_history": []})
    return response
demo = gr.ChatInterface(my_chat_function, chatbot=gr.Chatbot(height=300), theme="soft")
demo.launch()

Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


In [ ]:
from langchain.schema import format_document
from langchain_core.messages import get_buffer_string
from langchain_core.runnables import RunnableLambda

from langchain.prompts.prompt import PromptTemplate

from operator import itemgetter
from langchain.memory import ConversationBufferMemory


_template = """Given the following conversation and a follow up question, 
rephrase the follow up question to be a standalone question.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)


template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")

def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

memory = ConversationBufferMemory(
    return_messages=True, output_key="answer", input_key="question"
)

# First we add a step to load memory
# This adds a "memory" key to the input object
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)
# Now we calculate the standalone question
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: get_buffer_string(x["chat_history"]),
    }
    | CONDENSE_QUESTION_PROMPT
    | ChatOpenAI(temperature=0)
    | StrOutputParser(),
}

# Now we retrieve the documents
retrieved_documents = {
    "docs": itemgetter("standalone_question") | faiss_retriever,
    "question": lambda x: x["standalone_question"],
}


# Now we construct the inputs for the final prompt
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}
# And finally, we do the part that returns the answers
answer = {
    "answer": final_inputs | ANSWER_PROMPT | ChatOpenAI(), 
    "docs": itemgetter("docs"),
}
# And now we put it all together!
standalone_question = loaded_memory | standalone_question 

final_chain = standalone_question | retrieved_documents | answer

inputs = {"question": "What splitter is used to split the text?"}

result = final_chain.invoke(inputs)
print(result["answer"].content)
# Note that the memory does not save automatically
# This will be improved in the future
# For now you need to save it yourself
memory.save_context(inputs, {"answer": result["answer"].content})
memory.load_memory_variables({})

inputs = {"question": "Can you explain what this splitter does?"}
result = final_chain.invoke(inputs)
print(result["answer"].content)

In [ ]:
memory = ConversationBufferMemory(
    return_messages=True, output_key="answer", input_key="question"
)
def my_chat_function(message: str, history):
    input = {"question": message, }
    print(standalone_question.invoke(input))
    response = final_chain.invoke(input)
    memory.save_context(input, {"answer": response["answer"].content})
    memory.load_memory_variables({})

    return response["answer"].content
demo = gr.ChatInterface(my_chat_function, chatbot=gr.Chatbot(height=300), theme="soft")
demo.launch()